In [1]:
import datasets
from transformers import AutoTokenizer
import tqdm as notebook_tqdm
!pip install zstandard
from datasets import load_dataset
from transformers import AutoTokenizer

import json
import argparse
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments
import wandb
from transformers import LlamaForCausalLM, LlamaConfig 

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def tokenize(MAX_LENGTH=1024, OPTIMAL_NUM_TOKENS=64_000_000, SEED=4, dataset_path="cerebras/SlimPajama-627B", model_name="lmsys/vicuna-7b-v1.5"):
    # Load only testing data
    raw_datasets = load_dataset(dataset_path, streaming=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    shuffled_datasets = raw_datasets.shuffle(seed=SEED)

    tokenized_datasets = {}
    for key in raw_datasets.keys():
        tokenized_datasets[key] = shuffled_datasets[key].map(
            lambda x: tokenizer(
                x["text"],
                truncation=True,
                padding="max_length",
                max_length=MAX_LENGTH,
                return_tensors="pt",
            )
        )

    return tokenized_datasets

data = tokenize()

Resolving data files: 100%|██████████| 31411/31411 [00:00<00:00, 346325.15it/s]


In [3]:
next(iter(data['train']))

{'text': "I do think at this point you want Hoses item, so that you are within the proper weblog. Now you are examining BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU post. Along with Lets hope you will discover worthwhile details in below prior to buy Hoses product. At the moment I wish to reveal BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU. Little information that this BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU merchandise is done by means of BrassCraft Mfg firm.\nBrass Craft's 1/2 In. MIP EFV x 1/2 In. FIP x 72 In. Safety+PLUS Gas Appliance Connector is 1/2 In. OD (3/8 In. ID) with a maximum operating capacity of 49,100 BTU's. This Safety+PLUS gas connector combines excess flow technology with the advanced corrosion resistance of Recoat, to provide a

In [4]:
list(data['train'].take(1))

[{'text': "I do think at this point you want Hoses item, so that you are within the proper weblog. Now you are examining BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU post. Along with Lets hope you will discover worthwhile details in below prior to buy Hoses product. At the moment I wish to reveal BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU. Little information that this BrassCraft CSSD45E-72 P 1/2-Inch MIP EFV x 1/2-Inch FIP x 72 In.Safety+PLUS Gas Appliance Connector, 1/2-Inch OD 49,100 BTU merchandise is done by means of BrassCraft Mfg firm.\nBrass Craft's 1/2 In. MIP EFV x 1/2 In. FIP x 72 In. Safety+PLUS Gas Appliance Connector is 1/2 In. OD (3/8 In. ID) with a maximum operating capacity of 49,100 BTU's. This Safety+PLUS gas connector combines excess flow technology with the advanced corrosion resistance of Recoat, to provide 

In [5]:
with open('configs/config.json', 'r') as file:
    config_dict = json.load(file)
config = LlamaConfig.from_dict(config_dict)
llama2_model = LlamaForCausalLM(config)

In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    max_steps=10000,
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')
    
trainer = Trainer(
    model=llama2_model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    # tokenizer=tokenizer,
    
)

In [10]:
trainer.train()

eval_results = trainer.evaluate()

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Caught ValueError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/models/llama/modeling_llama.py", line 806, in forward
    outputs = self.model(
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/models/llama/modeling_llama.py", line 623, in forward
    batch_size, seq_length = input_ids.shape
ValueError: too many values to unpack (expected 2)


In [ ]:
data

In [2]:
MAX_LENGTH = 1024
OPTIMAL_NUM_TOKENS = 64_000_000
SEED = 42

In [3]:
raw_datasets = datasets.load_dataset("cerebras/SlimPajama-627B", streaming=True)
tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")

Resolving data files:   0%|          | 0/118332 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31428 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31411 [00:00<?, ?it/s]

In [4]:
shuffled_dataset = raw_datasets.shuffle(seed=SEED)

In [5]:
tokenized_dataset = {}
for key in raw_datasets.keys():
    tokenized_dataset[key] = shuffled_dataset[key].map(
        lambda x: tokenizer(
            x["text"],
            truncation=True,
            padding="max_length",
            max_length=MAX_LENGTH,
            return_tensors="pt",
        )
    )

In [6]:
for data in shuffled_dataset['train']:
    print(data)
    break

{'text': "If you were frustrated by third-party office application on your Symbian Belle phones, here's some good news for you. Nokia has just announced that users on their Belle platform can now user Microsoft Office Mobile apps on their phones now. This came out of the blue since most of us expected Microsoft's Office apps to be available exclusively on Nokia's Lumia phone series powered by Windows Phone. It's a good thing to see that Nokia-Microsoft partnership doesn't stop there, and is inclusive of Nokia's 'other' platform as well.\nMicrosoft Exchange Active Sync – To sync contacts, calendars and email from your company's Exchange Servers directly to your phone.\nMicrosoft Document Connect – To get access and work on your Word, PowerPoint and Excel files and access your company's SharePoint sites.\nMicrosoft One Note – allows you to create notes on your phone and sync them to Skydrive allowing you to access them from the PC as well.\nMicrosoft Lync – lets you join IM chats and aud

In [7]:
for data in tokenized_dataset['train']:
    print(data)
    break

{'text': "If you were frustrated by third-party office application on your Symbian Belle phones, here's some good news for you. Nokia has just announced that users on their Belle platform can now user Microsoft Office Mobile apps on their phones now. This came out of the blue since most of us expected Microsoft's Office apps to be available exclusively on Nokia's Lumia phone series powered by Windows Phone. It's a good thing to see that Nokia-Microsoft partnership doesn't stop there, and is inclusive of Nokia's 'other' platform as well.\nMicrosoft Exchange Active Sync – To sync contacts, calendars and email from your company's Exchange Servers directly to your phone.\nMicrosoft Document Connect – To get access and work on your Word, PowerPoint and Excel files and access your company's SharePoint sites.\nMicrosoft One Note – allows you to create notes on your phone and sync them to Skydrive allowing you to access them from the PC as well.\nMicrosoft Lync – lets you join IM chats and aud

In [8]:
list(tokenized_dataset['train'].take(1))

[{'text': "If you were frustrated by third-party office application on your Symbian Belle phones, here's some good news for you. Nokia has just announced that users on their Belle platform can now user Microsoft Office Mobile apps on their phones now. This came out of the blue since most of us expected Microsoft's Office apps to be available exclusively on Nokia's Lumia phone series powered by Windows Phone. It's a good thing to see that Nokia-Microsoft partnership doesn't stop there, and is inclusive of Nokia's 'other' platform as well.\nMicrosoft Exchange Active Sync – To sync contacts, calendars and email from your company's Exchange Servers directly to your phone.\nMicrosoft Document Connect – To get access and work on your Word, PowerPoint and Excel files and access your company's SharePoint sites.\nMicrosoft One Note – allows you to create notes on your phone and sync them to Skydrive allowing you to access them from the PC as well.\nMicrosoft Lync – lets you join IM chats and au